In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import numpy as np

In [2]:
random_state = 555

# Загрузка данных

In [3]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [4]:
train.isnull().sum()

ID                          0
join_date                   2
sex                         0
marital_status              0
birth_year                  0
branch_code                 0
occupation_code             0
occupation_category_code    0
P5DA                        0
RIBP                        0
8NN1                        0
7POT                        0
66FJ                        0
GYSR                        0
SOP4                        0
RVSZ                        0
PYUQ                        0
LJR9                        0
N2MW                        0
AHXO                        0
BSTQ                        0
FM3X                        0
K6QO                        0
QBOL                        0
JWFN                        0
JZ9D                        0
J9JW                        0
GHYX                        0
ECY3                        0
dtype: int64

In [5]:
test.isnull().sum()

ID                          0
join_date                   1
sex                         0
marital_status              0
birth_year                  0
branch_code                 0
occupation_code             0
occupation_category_code    0
P5DA                        0
RIBP                        0
8NN1                        0
7POT                        0
66FJ                        0
GYSR                        0
SOP4                        0
RVSZ                        0
PYUQ                        0
LJR9                        0
N2MW                        0
AHXO                        0
BSTQ                        0
FM3X                        0
K6QO                        0
QBOL                        0
JWFN                        0
JZ9D                        0
J9JW                        0
GHYX                        0
ECY3                        0
dtype: int64

Удалим из тренировочных данных пропуски, а в тестовом датафрейме заменим пропущенные данные

In [6]:
train = train.dropna()
test['join_date'] = test['join_date'].fillna('1/1/2018')

В тестовых данных вместо f проставлены F в marital status, приведем данные к одному формату

In [7]:
train['marital_status'] = train['marital_status'].replace('f', 'F')

# Трансформируем дату в timestamp

In [8]:
def tranforming_date(data):
    
    """
    Трансформирует датафрейм с заданной колонкой в unix timestamp

    ----------
    X_train - тренировочные данные
    y_train - тренировочный target
    X_test - тестовые данные
    model - модель с заданными параметрами
    ID_X_Var - датафрейм с ID пользователя и товара
    ----------

    """
    
    data['join_date'] = pd.to_datetime(data['join_date'], dayfirst = True)
    data['timestamp'] = data['join_date'].apply(lambda x: x.timestamp())
    
    return data

In [9]:
train = tranforming_date(train)
test = tranforming_date(test)

In [10]:
train = train.drop(['join_date'], axis = 'columns')
test = test.drop(['join_date'], axis = 'columns')

# Переведем target variables в строки

Сделаем такой датафейм, в котором каждое наблюдение будет соотвествовать паре пользователь - продукт. Также оставим данные о наличии остальных продуктов у пользователя. \
Таким образом, мы сохраним информацию о наличии каждого продукта у пользователя и будем предсказывать для каждого наблюдения лишь наличие одного конкретного продукта.

In [11]:
## Список переменных, которые надо предсказать
target_cols = ['P5DA', 'RIBP', '8NN1',
               '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
               'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']

In [12]:
## Переведем target variables в строки (широкий формат -> длинный формат)
train = pd.merge(train.melt(id_vars = [col for col in train.columns if col not in target_cols], 
                            value_vars = target_cols, 
                            value_name = 'presence'), 
                 train[['ID', *target_cols]], 
                 on = 'ID')

test = pd.merge(test.melt(id_vars = [col for col in test.columns if col not in target_cols], 
                          value_vars = target_cols, 
                          value_name = 'presence'), 
                 test[['ID', *target_cols]], 
                 on = 'ID')

# Кодирование переменных

In [13]:
cat_features = ['sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code']

def encoding(data_train, data_test, column):
    
    """
    Кодировка колонки датафрейма через LabelEncoder из sklearn

    ----------
    data_train - тренировочный датафрейм
    data_test - тестовый датафрейм
    column - колонка датафрейма, которую надо закодировать
    ----------

    """
    
    le = LabelEncoder()
    le.fit(np.append(data_train[column].values, data_test[column].values))
    data_train[column] = le.transform(data_train[column].values)
    data_test[column] = le.transform(data_test[column].values)
    return data_train, data_test

In [14]:
for column in cat_features:
    train, test = encoding(train, test, column)

# Создание тренировочного датасета

В тестовом датасете у нас есть множество пользователей, для которых нужно предсказать информацию о наличии каждого продукта. При этом у нас уже есть информация о наличии некоторых товаров у некоторых пользователей. \
Нам надо собрать такой тренировочный датасет, который максимально "повторит" тестовый. \
Для каждого отдельного фолда и для каждой его части (тренировочная/тестовая) будем для каждого пользователя делать следующее:
1. Выбираем все наблюдения для данного пользователя (то есть все пары пользователь - товар)
2. Выбираем все товары, которые уже есть у пользователя
3. Для каждого имеющегося товара создаем копию всех наблюдений пользователя, и удаляем из них информацию о наличии данного товара
4. Собираем вместе все датафрейми с прошлого шага

Таким образом, у нас получится датасет, в котором у нас будем информация о всех пользователях и наличии у них каждого товара. При этом мы искуственно "размножим" датасет поочередно убирая информацию о наличии у пользователя каждого товара и собирая вместе все эти датафреймы.

In [15]:
def reshaping(user_data):
    
    """
    Создает для пользователя все возможные комбинации, имеющихся у него товаров

    ----------
    user_data - данные о пользователе
    ----------

    """
    
    new_user_data = pd.DataFrame() # Датафрейм с новой информацией о пользователе
    user_data = user_data.reset_index(drop = True)
    #Выбираем все товары,которые есть у пользователя
    presence_1_vars = user_data[user_data['presence'] == 1]['variable'].values
    
    for var in presence_1_vars:
        temp_user_data = user_data.copy() # Копируем изначальные данные о пользователе
        temp_user_data[var] = 0 # Убираем информацию о наличии товара у пользователя
        new_user_data = new_user_data.append(temp_user_data, ignore_index = True)
    
    return(new_user_data)

In [16]:
train = train.groupby('ID').apply(reshaping)

In [17]:
train = train.reset_index(drop = True)

# Сохраним данные

In [18]:
train.to_csv('train_prepared.csv', index = False)
test.to_csv('test_prepared.csv', index = False)